In [58]:
from google.colab import drive
drive.mount('/content/drive')
# MeanImputation_df
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error,
    r2_score, mean_absolute_percentage_error
)
import plotly.express as px
import plotly.graph_objects as go

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
df = pd.read_csv("drive/My Drive/Master Thesis/main_feats.csv")
df.head()

,start_date,end_date,project_duration,x1,x2,x3,x4,x5,y
0,1396-01-01,1400-01-01,48,3150.0,920.0,1,1703.672462,16,9979.795043
1,1395-01-01,1400-10-01,69,7600.0,1140.0,1,1446.077707,23,16362.045150
2,1398-01-01,1401-10-01,45,4800.0,840.0,1,2376.845399,15,14120.093280
3,1397-04-01,1398-04-01,12,685.0,202.0,1,1207.741141,4,2560.672748
4,1387-01-01,1390-04-01,39,3000.0,800.0,1,410.000000,13,700.000000


In [60]:
df_economics = pd.read_csv("drive/My Drive/Master Thesis/Imputation/CubicImputation_df.csv")
df_economics.head()

,Date,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,Sin b6,Sin b7,Sin b8,Sin b9,Sin b10,Sin b14,Sin b17,Sin b18,Sin B-23,Sin B-24
0,1360-01-01,-0.312302,-0.573053,-1.008179,-0.486658,-0.519259,-0.550486,-0.524004,-0.474211,-0.763966,...,-0.760378,-0.519490,-0.536372,-0.881368,-0.592062,-0.674918,-0.894974,-0.765731,-0.739974,-0.624255
1,1360-04-01,-0.526017,-0.573053,-1.045895,-0.486639,-0.519352,-0.550486,-0.523908,-0.474211,-0.763704,...,-0.760378,-0.519405,-0.536372,-0.881702,-0.592062,-0.674918,-0.894543,-0.765731,-0.739974,-0.624255
2,1360-07-01,-0.431611,-0.573053,-1.097080,-0.486628,-0.519297,-0.550486,-0.523908,-0.474146,-0.763828,...,-0.760378,-0.519405,-0.536306,-0.881544,-0.592062,-0.674918,-0.894332,-0.765731,-0.739974,-0.624255
3,1360-10-01,-0.573509,-0.573053,-1.083610,-0.486563,-0.519403,-0.550486,-0.523716,-0.474211,-0.763741,...,-0.760378,-0.519234,-0.536372,-0.881655,-0.592062,-0.674918,-0.893711,-0.765731,-0.739974,-0.624255
4,1361-01-01,-0.586830,-0.573053,-1.038711,-0.486562,-0.518833,-0.550486,-0.523716,-0.474015,-0.763624,...,-0.760378,-0.519234,-0.536175,-0.881803,-0.590450,-0.674918,-0.894031,-0.765731,-0.739974,-0.624255


In [61]:
# filter df_economics drop extera rows for now (TODO: Add seasonal and  lag later)
df_economics = df_economics[df_economics['Date'].isin(df['start_date'])]
df = pd.merge(df, df_economics, how='left', left_on='start_date', right_on='Date')
del df_economics
df.drop(columns=['Date'], inplace=True)
df.drop(columns=['start_date', 'end_date'], inplace=True)
df = df[[c for c in df if c not in ['y']] + ['y']]
print(df.shape)
df.head()

(400, 52)


,project_duration,x1,x2,x3,x4,x5,b1,b2,b3,b4,...,Sin b7,Sin b8,Sin b9,Sin b10,Sin b14,Sin b17,Sin b18,Sin B-23,Sin B-24,y
0,48,3150.0,920.0,1,1703.672462,16,-0.627952,0.512964,-0.215028,0.971098,...,0.473973,0.263595,-9.090529e-01,0.294251,-0.124358,0.975063,0.964070,0.954525,0.769512,9979.795043
1,69,7600.0,1140.0,1,1446.077707,23,-0.903059,0.432169,0.287803,0.687847,...,0.383201,0.210828,-6.687348e-01,0.225918,-0.221708,0.999795,0.905790,0.901170,0.658368,16362.045150
2,45,4800.0,840.0,1,2376.845399,15,-0.660964,1.126800,-0.171924,1.708586,...,0.929428,0.999566,-1.262291e-01,0.307541,0.975338,-0.041006,0.865552,0.931351,0.889746,14120.093280
3,12,685.0,202.0,1,1207.741141,4,-0.268284,0.718625,0.329425,1.367522,...,0.653622,0.586774,-2.449294e-16,0.756817,0.921560,0.433860,0.996015,0.999972,0.954203,2560.672748
4,39,3000.0,800.0,1,410.000000,13,1.995128,-0.328568,1.600488,-0.307190,...,-0.258078,-0.278387,-2.820141e-01,-0.397584,-0.609905,-0.503466,-0.134573,-0.134950,-0.311864,700.000000


In [62]:
model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.9,
    random_state=42,
    early_stopping_rounds=50,
)

x = df[[c for c in df if c not in ['y']]]
y = df['y']

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_valid, x_test = x.iloc[:int(len(x)*0.7)], x.iloc[int(len(x)*0.7):int(len(x)*0.9)], x.iloc[int(len(x)*0.9):]
y_train, y_valid, y_test = y.iloc[:int(len(y)*0.7)], y.iloc[int(len(y)*0.7):int(len(y)*0.9)], y.iloc[int(len(y)*0.9):]
model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=100)

[0]	validation_0-rmse:3599.58687
[100]	validation_0-rmse:771.18360
[200]	validation_0-rmse:759.41075
[300]	validation_0-rmse:754.08423
[307]	validation_0-rmse:754.14161


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

In [63]:
# plot training and validation accuracy and loss
results = model.evals_result()
epochs = len(results['validation_0']['rmse'])
x_axis = [i for i in range(0, epochs)]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_axis, y=results['validation_0']['rmse'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=x_axis, y=results['validation_0']['rmse'], mode='lines', name='Valid'))
fig.update_layout(title='Training and Validation RMSE', xaxis_title='Epochs', yaxis_title='RMSE')
fig.show()

In [64]:
# evaluate the model
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print('MSE: %.3f, MAE: %.3f, R2: %.3f, MAPE: %.3f' % (mse, mae, r2, mape))

MSE: 35170514.017, MAE: 2550.448, R2: 0.702, MAPE: 1.598
